In [13]:
%matplotlib inline

In [23]:
import matplotlib.pyplot as plt
import obspy
import os
import numpy as np
from obspy.io.sac.sactrace import SACTrace
from obspy.clients.fdsn import Client as irisClient
from obspy.clients.arclink.client import Client
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
from obspy.io.xseed import Parser
from obspy.core.inventory import Inventory, Network, Station, Channel, Site

### Coletando os dados dos Eventos

In [24]:
#NEIC 2016-2017
NEIC  = '/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/codes_RF_process_BP/FR_programs/neic.BB'

In [25]:
event_info = np.genfromtxt(NEIC,delimiter=',',skip_header=1,usecols=[1,2,3,4])

In [26]:

#Guardando as variáves cada evento:

ev_time =  np.genfromtxt(NEIC,delimiter=',',dtype='str',skip_header=1,usecols=[0])

ev_timeUTC = []
ev_year = []
ev_month = []
ev_day = []
ev_julday = []
ev_hour = []
ev_minute = []
ev_second = []
ev_microsecond = []

for i in ev_time:
    temp = obspy.UTCDateTime(i)
    ev_year.append(str(temp.year))
    ev_month.append(str("{0:02.0f}".format(temp.month)))
    ev_julday.append(str(temp.julday))
    ev_day.append(str("{0:02.0f}".format(temp.day)))
    ev_hour.append(str(temp.hour))
    ev_minute.append(str(temp.minute))
    ev_second.append(str(temp.second))
    ev_microsecond.append(str(temp.microsecond))
    ev_timeUTC.append(obspy.UTCDateTime(i))

ev_lat = event_info[:,0]
ev_long = event_info[:,1]
ev_depth = event_info[:,2]
ev_mag = event_info[:,3]

### Baixando os dados paras a estação SMTB

In [27]:
sta_lat_SMTB = -8.8617
sta_long_SMTB = -47.5886

In [28]:
event_starttime_SMTB = []
event_endtime_SMTB = []

In [33]:
dist_lst = []
az_lst = []
baz_lst = []
gcarc_lst = []
user8_lst = []
for i,j in enumerate(ev_timeUTC):
        dist, az, baz = obspy.geodetics.gps2dist_azimuth(ev_lat[i], ev_long[i], sta_lat_SMTB,sta_long_SMTB)
        if obspy.geodetics.kilometer2degrees(dist/1000) >= 30 and obspy.geodetics.kilometer2degrees(dist/1000) <= 90:
            dist_lst.append(dist)
            az_lst.append(az)
            baz_lst.append(baz)
            gcarc = obspy.geodetics.kilometer2degrees(dist/1000)
            gcarc_lst.append(gcarc)
            model = TauPyModel(model="iasp91")
            arrivals = model.get_travel_times(source_depth_in_km=ev_depth[i],distance_in_degree=gcarc, phase_list=["P"])
            arr = arrivals[0]
            user8_lst.append(arr.ray_param/6371)
            event_starttime_SMTB.append(ev_timeUTC[i]+arr.time-500)
            event_endtime_SMTB.append(ev_timeUTC[i]+arr.time+(3500))

In [9]:
loc_folder = '/home/diogo/dados_doutorado/parnaiba_basin/assessing_data/data_7mb/event_data/'

In [35]:
client = Client(user='locdiogo@gmail.com',host='10.3.227.41', port=18001, institution='UFRN',timeout=200)

ArcLinkException: Timeout waiting for expected b')', got 

In [11]:
st_SMTB_Z = []
st_SMTB_Z = [client.get_waveforms('BR','SMTB','','HHZ',j, event_endtime_SMTB[i]) for i,j in enumerate(event_starttime_SMTB)]

ArcLinkException: No data available

In [ ]:
st_SMTB_N = []

st_SMTB_N = [client.get_waveforms('BR','SMTB','','HHN',j, event_endtime_SMTB[i]) for i,j in enumerate(event_starttime_SMTB)]

In [ ]:
st_SMTB_E = []

st_SMTB_E = [client.get_waveforms('BR','SMTB','','HHE',j, event_endtime_SMTB[i]) for i,j in enumerate(event_starttime_SMTB)]

In [ ]:
for k,l in enumerate(st_SMTB_Z):
    try:
        j = l[0].stats.starttime
        folder_loc_string = loc_folder+'SMTB'+'/'+str(j.year)+'/'+str("{0:0=3d}".format(j.julday))+'/'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)
        os.makedirs(folder_loc_string,exist_ok=True)
        headerHHZ = {'kstnm': 'SMTB', 'kcmpnm': 'HHZ', 'stla': sta_lat_SMTB, 'stlo': sta_long_SMTB, 'evdp': ev_depth[k],  
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]),'nzmsec': float(ev_microsecond[k])/10000, 'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 0.0, 'cmpinc': 0.0, 'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k], 
                     'user8': user8_lst[k],'delta':l[0].stats.delta,'npts':l[0].stats.npts}
        sacHHZ = SACTrace(data=l[0].data, **headerHHZ)
        sacHHZ.write(folder_loc_string+'/BR.SMTB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.z')
        
        headerHHY = {'kstnm': 'SMTB', 'kcmpnm': 'HHN', 'stla': sta_lat_SMTB, 'stlo': sta_long_SMTB, 'evdp': ev_depth[k],
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]), 'nzmsec': float(ev_microsecond[k])/10000,'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 0.0, 'cmpinc': 90.0, 'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k],
                     'user8': user8_lst[k],'delta':st_SMTB_N[k][0].stats.delta,'npts':st_SMTB_N[k][0].stats.npts}
        sacHHY = SACTrace(data=st_SMTB_N[k][0].data, **headerHHY)
        sacHHY.write(folder_loc_string+'/BR.SMTB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.n')
        
        headerHHX = {'kstnm': 'SMTB', 'kcmpnm': 'HHE', 'stla': sta_lat_SMTB, 'stlo': sta_long_SMTB, 'evdp': ev_depth[k],  
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]),
                     'nzmsec': float(ev_microsecond[k])/10000, 'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 90.0, 'cmpinc': 90.0,'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k], 
                    'user8': user8_lst[k],'delta':st_SMTB_E[k][0].stats.delta,'npts':st_SMTB_E[k][0].stats.npts}
        sacHHX = SACTrace(data=st_SMTB_E[k][0].data, **headerHHX)
        sacHHX.write(folder_loc_string+'/BR.SMTB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.e')

    except:
        print('Deu RUIM')

### Baixando os dados paras a estação ROSB

In [ ]:
sta_lat_ROSB = -2.8967
sta_long_ROSB = -44.1246

In [ ]:
event_starttime_ROSB = []
event_endtime_ROSB = []

In [ ]:
dist_lst = []
az_lst = []
baz_lst = []
gcarc_lst = []
user8_lst = []
for i,j in enumerate(ev_timeUTC):
        dist, az, baz = obspy.geodetics.gps2dist_azimuth(ev_lat[i], ev_long[i], sta_lat_ROSB,sta_long_ROSB)
        dist_lst.append(dist)
        az_lst.append(az)
        baz_lst.append(baz)
        gcarc = obspy.geodetics.kilometer2degrees(dist/1000)
        gcarc_lst.append(gcarc)
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=ev_depth[i],distance_in_degree=gcarc, phase_list=["P"])
        arr = arrivals[0]
        user8_lst.append(arr.ray_param/6371)
        event_starttime_ROSB.append(ev_timeUTC[i]+arr.time-500)
        event_endtime_ROSB.append(ev_timeUTC[i]+arr.time+(3500))

In [ ]:
st_ROSB_Z = []

st_ROSB_Z = [client.get_waveforms('BR','ROSB','','HHZ',j, event_endtime_ROSB[i]) for i,j in enumerate(event_starttime_ROSB)]

In [ ]:
st_ROSB_N = []

st_ROSB_N = [client.get_waveforms('BR','ROSB','','HHN',j, event_endtime_ROSB[i]) for i,j in enumerate(event_starttime_ROSB)]

In [ ]:
st_ROSB_E = []

st_ROSB_E = [client.get_waveforms('BR','ROSB','','HHE',j, event_endtime_ROSB[i]) for i,j in enumerate(event_starttime_ROSB)]

In [ ]:
for k,l in enumerate(st_ROSB_Z):
    try:
        j = l[0].stats.starttime
        folder_loc_string = loc_folder+'ROSB'+'/'+str(j.year)+'/'+str("{0:0=3d}".format(j.julday))+'/'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)
        os.makedirs(folder_loc_string,exist_ok=True)
        headerHHZ = {'kstnm': 'ROSB', 'kcmpnm': 'HHZ', 'stla': sta_lat_ROSB, 'stlo': sta_long_ROSB, 'evdp': ev_depth[k],  
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]),'nzmsec': float(ev_microsecond[k])/10000, 'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 0.0, 'cmpinc': 0.0, 'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k], 
                     'user8': user8_lst[k],'delta':l[0].stats.delta,'npts':l[0].stats.npts}
        sacHHZ = SACTrace(data=l[0].data, **headerHHZ)
        sacHHZ.write(folder_loc_string+'/BR.ROSB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.z')
        
        headerHHY = {'kstnm': 'ROSB', 'kcmpnm': 'HHN', 'stla': sta_lat_ROSB, 'stlo': sta_long_ROSB, 'evdp': ev_depth[k],
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]), 'nzmsec': float(ev_microsecond[k])/10000,'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 0.0, 'cmpinc': 90.0, 'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k],
                     'user8': user8_lst[k],'delta':st_ROSB_N[k][0].stats.delta,'npts':st_ROSB_N[k][0].stats.npts}
        sacHHY = SACTrace(data=st_ROSB_N[k][0].data, **headerHHY)
        sacHHY.write(folder_loc_string+'/BR.ROSB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.n')
        
        headerHHX = {'kstnm': 'ROSB', 'kcmpnm': 'HHE', 'stla': sta_lat_ROSB, 'stlo': sta_long_ROSB, 'evdp': ev_depth[k],  
                     'evla': ev_lat[k], 'evlo': ev_long[k], 'mag': ev_mag[k], 'nzhour': int(ev_hour[k]),
                     'nzjday': int(ev_julday[k]), 'nzmin': int(ev_minute[k]),
                     'nzmsec': float(ev_microsecond[k])/10000, 'nzsec': int(ev_second[k]), 'nzyear': int(ev_year[k]),
                     'cmpaz': 90.0, 'cmpinc': 90.0,'dist': dist_lst[k]/1000, 'gcarc': gcarc_lst[k], 'az': az_lst[k], 'baz': baz_lst[k], 
                     'user8': user8_lst[k],'delta':st_ROSB_E[k][0].stats.delta,'npts':st_ROSB_E[k][0].stats.npts}
        sacHHX = SACTrace(data=st_ROSB_E[k][0].data, **headerHHX)
        sacHHX.write(folder_loc_string+'/BR.ROSB.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+str(j.microsecond)+'.e')

    except:
        print('Deu RUIM')